---
title: Métricas de Equidade em Aprendizado de Máquina e seus Fundamentos Éticos e Políticos
author: Fernando Pasquini Santos - Department of Computer Science, Calvin University
format:
    revealjs:
        selfcontained: false
        width: 1200
        slide-number: true
        max-scale: 2
        slide-level: 2
        scrollable: true
execute:
  echo: true
include-in-header:
  - text: |
      <style>
      #title-slide .title {
        font-size: 2em;;
      }
      .small-text {
      font-size: 0.7em; /* Adjust the value to your desired text size */
      }
      </style>
---


## Sociedade da competição

Pressuposto: um mundo de recursos limitados e, portanto, de competição em todos os lugares e todas as áreas.

- em empregos/admissões
- em provas/testes/avaliações
- em honras/medalhas/etc
- em dinheiro/crédito

*"O mundo é feito de ganhadores e perdedores"*

- Justiça, então, seria garantir que a competição é justa ("fair game")

## Inequidade e discriminação

Três possíveis sentidos:

1. Atribuir vantagem/desvantagem a alguém com base em **critérios errados de relevância**
2. Atribuir vantagem/desvantagem a alguém com base em **estereótipos e generalizações**
3. Atribuir vantagem/desvantagem a alguém com base em **coisas sobre as quais a pessoa não tem controle**

## Automação e burocracia

Por que buscamos?

1. Automação permite escalar o sistema
2. Regras e procedimentos formais, *em teoria*, reduzem a arbitrariedade e tornam o raciocínio explícito.
    - Supõem de que o processo apenas envolve conhecimento explícito (e não tácito)

## Automação e burocracia

No caso do uso de Machine Learning: "decisão baseada em dados"

  - Parece algo formal, mas o raciocínio acaba sendo uma caixa-preta (opacidade epistêmica)
  - Dados introduzem problemas:
    - Arbitrariedade na anotação dos dados (*target labeling*)
    - Arbitrariedade na seleção dos atributos/*features*
    - Limite da indução
  - Taxa de erros: maior ou menor?
  - Responsabilização e possibilidade de recursos?

## Nosso exemplo: conjunto de dados ficcional {.smaller}

- 1.000 linhas, onde cada linha representa o perfil de um estudante, com atributos relacionados ao desempenho acadêmico e não acadêmico, necessidade financeira e informações demográficas.
- Informações são geralmente consideradas em decisões sobre concessão de bolsas de estudo.

1. **`Student_ID`**: um identificador único para cada estudante.
2. **`GPA`**: média de notas, variando de **2.0** a **4.0**.
3. **`Extracurricular_Score`**: uma pontuação que representa a participação e conquistas do estudante em atividades extracurriculares, variando de **0** a **100**.
4. **`Community_Service_Hours`**: o total de horas que o estudante dedicou a atividades de serviço comunitário, variando de **0** a **200**.
5. **`Financial_Need`**: indica se o estudante possui necessidade financeira.
6. **`Scholarship_Granted`**: indica se o estudante recebeu bolsa de estudo.
7. **`Sensitive_Attribute`**: um atributo sensível genérico com dois grupos, usado para análise de equidade (por exemplo, gênero, cor, idade, etc).


In [ ]:
import pandas as pd

df = pd.read_csv('https://cs.calvin.edu/courses/data/202/fsantos/datasets/student_performance.csv')
df.loc[:,'Sensitive_Attribute'] = df['Sensitive_Attribute'].apply(lambda x: 0 if x == 'Group A' else 1)
display(df)

## Treinando o modelo, considerando o atributo sensível


In [ ]:
#| echo: true
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Define features (X) and target variable (y)
X = df[['GPA', 'Extracurricular_Score', 'Community_Service_Hours', 'Financial_Need', 'Sensitive_Attribute']]
y = df['Scholarship_Granted']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train a Logistic Regression model
model = LogisticRegression()
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

---

### Matriz de confusão {.smaller}

|                           | **Classe Predita: Positiva** | **Classe Predita: Negativa** |
| ------------------------- | ---------------------------- | ---------------------------- |
| **Classe Real: Positiva** | Verdadeiro Positivo (VP)     | Falso Negativo (FN)          |
| **Classe Real: Negativa** | Falso Positivo (FP)          | Verdadeiro Negativo (VN)     |

---

### Métricas {.smaller}

| Métrica                                  | Fórmula                                             | Interpretação                                                      |
| ---------------------------------------- | --------------------------------------------------- | ------------------------------------------------------------------ |
| **Acurácia**                             | (VP + VN) / Total                                   | Proporção de acertos totais                                        |
| **Precisão (Positive Predictive Value)** | VP / (VP + FP)                                      | Entre os classificados como positivos, quantos realmente são       |
| **Revocação (Recall ou Sensibilidade)**  | VP / (VP + FN)                                      | Entre os positivos reais, quantos foram corretamente identificados |
| **Especificidade**                       | VN / (VN + FP)                                      | Entre os negativos reais, quantos foram corretamente identificados |
| **Taxa de Falsos Positivos (FPR)**       | FP / (FP + VN)                                      | Negativos reais classificados erroneamente como positivos          |
| **Taxa de Falsos Negativos (FNR)**       | FN / (FN + VP)                                      | Positivos reais classificados erroneamente como negativos          |
| **F1 Score**                             | 2 × (Precisão × Revocação) / (Precisão + Revocação) | Média harmônica entre precisão e revocação                         |
| **Valor Preditivo Negativo (NPV)**       | VN / (VN + FN)                                      | Entre os classificados como negativos, quantos realmente são       |
| **Taxa de Prevalência**                  | (VP + FN) / Total                                   | Proporção de positivos na população real                           |

## No nosso caso:


In [ ]:
#| echo: false
from sklearn.metrics import confusion_matrix

# Calculate the confusion matrix for the entire test set
cm_overall = confusion_matrix(y_test, y_pred)

# Separate the test set by the sensitive attribute
X_test_groupA = X_test[X_test['Sensitive_Attribute'] == 0]
y_test_groupA = y_test[X_test['Sensitive_Attribute'] == 0]
y_pred_groupA = model.predict(X_test_groupA)

X_test_groupB = X_test[X_test['Sensitive_Attribute'] == 1]
y_test_groupB = y_test[X_test['Sensitive_Attribute'] == 1]
y_pred_groupB = model.predict(X_test_groupB)

# Calculate the confusion matrix for Group A
cm_groupA = confusion_matrix(y_test_groupA, y_pred_groupA)

# Calculate the confusion matrix for Group B
cm_groupB = confusion_matrix(y_test_groupB, y_pred_groupB)

# Create DataFrames for better readability
cm_overall_df = pd.DataFrame(cm_overall, index=['Actual 0', 'Actual 1'], columns=['Predicted 0', 'Predicted 1'])
cm_groupA_df = pd.DataFrame(cm_groupA, index=['Actual 0', 'Actual 1'], columns=['Predicted 0', 'Predicted 1'])
cm_groupB_df = pd.DataFrame(cm_groupB, index=['Actual 0', 'Actual 1'], columns=['Predicted 0', 'Predicted 1'])

print("\nOverall Confusion Matrix:")
display(cm_overall_df)
print("\nConfusion Matrix for Group A:")
display(cm_groupA_df)
print("\nConfusion Matrix for Group B:")
display(cm_groupB_df)

---


In [ ]:
#| echo: false

def create_metrics_table(y_true, y_pred, dataframe, group_col_name='Group'):
    """
    Creates a formatted table showing various classification metrics per group.

    Args:
        y_true (pd.Series): True labels.
        y_pred (pd.Series): Predicted labels.
        groups (pd.Series): Series indicating the group for each data point.
        group_col_name (str): The name to use for the group column in the output table.

    Returns:
        pd.DataFrame: A DataFrame containing the metrics per group.
    """
    metrics_data = []
    groups = dataframe.loc[y_true.index, 'Sensitive_Attribute']
    unique_groups = groups.unique()

    for group_value in unique_groups:
        group_name = f"{group_col_name} B" if group_value else f"{group_col_name} A"
        group_indices = groups == group_value

        y_true_group = y_true[group_indices]
        y_pred_group = y_pred[group_indices]

        if len(y_true_group) == 0:
            metrics_data.append({group_col_name: group_name,
                                 'Acceptance Rate': np.nan,
                                 'Accuracy': np.nan,
                                 'Precision': np.nan,
                                 'Recall': np.nan,
                                 'F1-Score': np.nan})
            continue

        cm_group = confusion_matrix(y_true_group, y_pred_group)

        # Calculate metrics
        tn, fp, fn, tp = cm_group.ravel()
        total_samples = len(y_true_group)

        # Acceptance Rate (proportion of positive predictions)
        acceptance_rate = f"{fp + tp}/{total_samples}, {(100*(fp + tp)/total_samples):.2f}%"

        # Accuracy
        accuracy = (tn + tp) / total_samples if total_samples > 0 else 0

        # Precision (avoid division by zero if no positive predictions)
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0

        # Recall (Sensitivity or True Positive Rate) (avoid division by zero if no actual positives)
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0

        # F1-Score (avoid division by zero if precision and recall are zero)
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

        metrics_data.append({group_col_name: group_name,
                             'Acceptance Rate': acceptance_rate,
                             'Accuracy': f"{accuracy*100:.1f}%",
                             'Precision': f"{precision*100:.1f}%",
                             'Recall': f"{recall*100:.1f}%",
                             'F1-Score': f"{f1*100:.1f}%"})

    metrics_df = pd.DataFrame(metrics_data)
    metrics_df = metrics_df.set_index('Group')
    metrics_df.index.name = 'Metric'
    metrics_df = metrics_df.transpose().reindex(columns=['Group A', 'Group B'])
    return metrics_df

# Create the metrics table
metrics_table = create_metrics_table(y_test, y_pred, df, group_col_name='Group')
display(metrics_table)

## Em que momento considerar a equidade?

* Pré-processamento: ajustar o espaço de atributos para que não tenha correlação com o atributo sensível.
* Durante o treinamento: incorporar a restrição no processo de otimização que constrói o classificador.
* Pós-processamento: ajustar um classificador já treinado para que não tenha correlação com o atributo sensível.

Vamos focar no pós-processamento, considerando que o atributo sensível foi eliminado no treinamento.

## Quatro estratégias

Se resolvermos eliminar o atributo sensível do treinamento:

1. Ignorar os grupos, usar o mesmo limiar de decisão;
2. Usar limiares de decisão diferentes para maximizar a acurácia ("máximo lucro")
3. Paridade demográfica: alcançar uma taxa de aceitação igual em todos os grupos;
4. Paridade de erro ou acurácia: alcançar taxas iguais de verdadeiros/falsos positivos/negativos em todos os grupos.

## 1a estratégia: ignorar os grupos, limiar único


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Define features (X) and target variable (y) - ignore sensitive attribute
X = df[['GPA', 'Extracurricular_Score', 'Community_Service_Hours', 'Financial_Need']]
y = df['Scholarship_Granted']

# Convert 'Financial_Need' to numerical (0 or 1) if it's not already
if not pd.api.types.is_numeric_dtype(X['Financial_Need']):
    X['Financial_Need'] = X['Financial_Need'].astype(int)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train a Logistic Regression model
model = LogisticRegression()
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

---

### Problema 1: Métricas ainda continuam diferentes


In [ ]:
metrics_table = create_metrics_table(y_test, y_pred, df, group_col_name='Group')
display(metrics_table)

In [ ]:
#| echo: false

import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def plot_probability_with_jitter_by_group(df_proba, thresholds=[0.5, 0.5], jitter_strength=0.2):
    """
    Plots predicted probabilities by true class with horizontal jitter, separated by sensitive attribute group.

    Parameters:
        df_proba (pd.DataFrame): DataFrame containing 'Prob_Class_1' (predicted probabilities),
                                 'True_Label' (actual class labels), and 'Sensitive_Attribute' (group).
        threshold (float): Decision threshold to visualize on the plot.
        jitter_strength (float): The maximum amount of horizontal jitter to apply to scatter points.

    Returns:
        go.Figure: The Plotly figure object with subplots.
    """
    # Validate input columns
    if 'Prob_Class_1' not in df_proba.columns or 'True_Label' not in df_proba.columns or 'Sensitive_Attribute' not in df_proba.columns:
        raise ValueError("Input DataFrame must contain 'Prob_Class_1', 'True_Label', and 'Sensitive_Attribute' columns.")

    unique_groups = df_proba['Sensitive_Attribute'].unique()
    unique_groups.sort()
    num_groups = len(unique_groups)
    group_names = {0: 'Group A', 1: 'Group B'}

    # Create subplots: one row, multiple columns based on the number of unique groups
    fig = make_subplots(rows=1, cols=num_groups,
                        subplot_titles=[group_names.get(group, f'Group {group}') for group in unique_groups],
                        horizontal_spacing=0.1) # Adjust spacing between subplots if needed

    for i, group in enumerate(unique_groups):
        col_index = i + 1 # Plotly columns are 1-indexed

        df_group = df_proba[df_proba['Sensitive_Attribute'] == group].copy()

        if len(df_group) == 0:
            continue

        # Add jitter to the probabilities
        df_group['Jitter'] = np.random.uniform(-jitter_strength/2, jitter_strength/2, size=len(df_group))

        # Add scatter plot for Class 0 in the group
        fig.add_trace(go.Scattergl(
            x=df_group[df_group['True_Label'] == 0]['Jitter'],
            y=df_group[df_group['True_Label'] == 0]['Prob_Class_1'],
            mode='markers',
            name=f"True Class 0 (Reject)",
            marker=dict(color="red", opacity=0.5, size=6),
            text=df_group[df_group['True_Label'] == 0].index,
            hoverinfo='text',
            showlegend= (i == 0) # Only show legend for the first subplot to avoid redundancy
        ), row=1, col=col_index)

        # Add scatter plot for Class 1 in the group
        fig.add_trace(go.Scattergl(
            x=df_group[df_group['True_Label'] == 1]['Jitter'],
            y=df_group[df_group['True_Label'] == 1]['Prob_Class_1'],
            mode='markers',
            name=f"True Class 1 (Accept)",
            marker=dict(color="blue", opacity=0.5, size=6),
            text=df_group[df_group['True_Label'] == 1].index,
            hoverinfo='text',
             showlegend= (i == 0) # Only show legend for the first subplot
        ), row=1, col=col_index)

        # Add the threshold line for each subplot
        fig.add_hline(
            y=thresholds[i],
            line_dash="dash",
            line_color="black",
            annotation_text=f"Threshold = {thresholds[i]}",
            annotation_position="top right",
            row=1, col=col_index
        )

        # Update layout for each subplot's axes
        fig.update_xaxes(showticklabels=False, row=1, col=col_index) # Hide x-axis tick labels
        fig.update_yaxes(title_text="Predicted Probability of Class 1", row=1, col=col_index)

    # Update global layout
    fig.update_layout(
        legend_title="True Label",
        height=500, # Adjust height as needed
        width=800 # Adjust width as needed
    )

    # Return the plot
    return fig

In [ ]:
#| echo: false
proba_scores = model.predict_proba(X_test)
df_proba = pd.DataFrame(proba_scores, columns=['Prob_Class_0', 'Prob_Class_1'])
df_proba['True_Label'] = y_test.values
df_proba['Sensitive_Attribute'] = df.iloc[y_test.index]['Sensitive_Attribute'].values

fig = plot_probability_with_jitter_by_group(df_proba)
fig.show()

---

### Problema 2: Discriminação por *proxy*

- Outros atributos podem refletir os grupos.
- Por exemplo, vamos identificar o atributo com maior importância:


In [ ]:
# Get feature importances from the trained model
feature_importances = model.coef_[0]

# Create a DataFrame to display feature importances
feature_importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': feature_importances})

# Sort the DataFrame by importance
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# Print the feature importances
feature_importance_df

---

- Observe como o GPA é afetado pelos grupos:


In [ ]:
import plotly.express as px

# Group data by 'Sensitive_Attribute' and calculate the mean GPA for each group
gpa_by_group = df.groupby('Sensitive_Attribute')['GPA'].mean()

# Create a bar plot using Plotly Express
fig = px.bar(gpa_by_group, x=gpa_by_group.index, y='GPA',
             labels={'GPA': 'Mean GPA', 'Sensitive_Attribute': 'Group'},
             title='Mean GPA by Sensitive Attribute')
fig.show()

## 2a estratégia: diferentes limiares, máximo lucro

Do ponto de vista de quem vai ofertar as bolsas, a ideia é acertar o máximo que puder.

E se limiares diferentes forem atribuídos aos grupos de forma a maximizar os acertos?


In [ ]:
# Find the optimal threshold for Group A to minimize errors
best_threshold_A = 0
min_errors_A = float('inf')

# Filter for groups
df_proba_group_A = df_proba[df_proba['Sensitive_Attribute'] == 0]
df_proba_group_B = df_proba[df_proba['Sensitive_Attribute'] == 1]

for threshold in np.linspace(0, 1, 101):  # Test thresholds from 0 to 1
    df_proba_group_A_test = df_proba_group_A.copy()
    df_proba_group_A_test['Predicted_Label'] = (df_proba_group_A['Prob_Class_1'] >= threshold).astype(int)
    errors_A = sum(df_proba_group_A_test['Predicted_Label'] != df_proba_group_A_test['True_Label'])
    if errors_A < min_errors_A:
        min_errors_A = errors_A
        best_threshold_A = threshold

print(f"Best threshold for Group A: {best_threshold_A}")
print(f"Minimum errors for Group A: {min_errors_A}")

# Find the optimal threshold for Group B to minimize errors
best_threshold_B = 0
min_errors_B = float('inf')

for threshold in np.linspace(0, 1, 101):  # Test thresholds from 0 to 1
    df_proba_group_B_test = df_proba_group_B.copy()
    df_proba_group_B_test['Predicted_Label'] = (df_proba_group_B['Prob_Class_1'] >= threshold).astype(int)
    errors_B = sum(df_proba_group_B_test['Predicted_Label'] != df_proba_group_B_test['True_Label'])
    if errors_B < min_errors_B:
        min_errors_B = errors_B
        best_threshold_B = threshold

print(f"Best threshold for Group B: {best_threshold_B}")
print(f"Minimum errors for Group B: {min_errors_B}")

---


In [ ]:
#| echo: false
fig = plot_probability_with_jitter_by_group(df_proba, thresholds=[best_threshold_A, best_threshold_B])
fig.show()

In [ ]:
#| echo: false
# Get y_pred based on the group-specific thresholds
y_pred_adjusted = np.zeros_like(y_test) # Initialize with zeros (default class 0)

# Apply threshold for Group A
group_A_indices_test = df.loc[y_test.index, 'Sensitive_Attribute'] == 0
y_pred_adjusted[group_A_indices_test] = (df_proba_group_A['Prob_Class_1'] >= best_threshold_A).astype(int)

# Apply threshold for Group B
group_B_indices_test = df.loc[y_test.index, 'Sensitive_Attribute'] == 1
y_pred_adjusted[group_B_indices_test] = (df_proba_group_B['Prob_Class_1'] >= best_threshold_B).astype(int)

metrics_table_adjusted = create_metrics_table(y_test, y_pred_adjusted, df, group_col_name='Group')
display(metrics_table_adjusted)

## 3a estratégia: paridade demográfica

- A mesma quantidade de pessoas de cada grupo é escolhida (Positivos A = Positivos B)


In [ ]:
# Find thresholds to achieve demographic parity
def find_thresholds_for_demographic_parity(df_group_A, df_group_B, lambda_weight=0.5):
    best_threshold_A = 0
    best_threshold_B = 0
    best_combined_metric = float('inf')

    def calculate_accuracy(df, threshold):
        df_test = df.copy()
        df_test['Predicted_Label'] = (df_test['Prob_Class_1'] >= threshold).astype(int)
        accuracy = (df_test['Predicted_Label'] == df_test['True_Label']).mean()
        return accuracy

    # Test thresholds for both groups
    for threshold_A in np.linspace(0, 1, 101):
        for threshold_B in np.linspace(0, 1, 101):
            ppr_A = df_proba_group_A[df_group_A['Prob_Class_1'] >= threshold_A].shape[0]
            ppr_B = df_proba_group_B[df_group_B['Prob_Class_1'] >= threshold_B].shape[0]
            ppr_diff = abs(ppr_A - ppr_B)

            # Calculate accuracy for both groups
            accuracy_A = calculate_accuracy(df_group_A, threshold_A)
            accuracy_B = calculate_accuracy(df_group_B, threshold_B)
            overall_accuracy = (accuracy_A + accuracy_B) / 2

            # Combine PPR difference and accuracy into a single metric
            combined_metric = lambda_weight * ppr_diff - (1 - lambda_weight) * overall_accuracy

            # Update best thresholds if the combined metric improves
            if combined_metric < best_combined_metric:
                best_combined_metric = combined_metric
                best_threshold_A = threshold_A
                best_threshold_B = threshold_B

    return best_threshold_A, best_threshold_B

# Find the thresholds for demographic parity
best_threshold_A, best_threshold_B = find_thresholds_for_demographic_parity(df_proba_group_A, df_proba_group_B)

print(f"Threshold for Group A (Demographic Parity): {best_threshold_A}")
print(f"Threshold for Group B (Demographic Parity): {best_threshold_B}")

---


In [ ]:
#| echo: false
fig = plot_probability_with_jitter_by_group(df_proba, thresholds=[best_threshold_A, best_threshold_B])
fig.show()

In [ ]:
#| echo: false
# Get y_pred based on the group-specific thresholds
y_pred_adjusted = np.zeros_like(y_test) # Initialize with zeros (default class 0)

# Apply threshold for Group A
group_A_indices_test = df.loc[y_test.index, 'Sensitive_Attribute'] == 0
y_pred_adjusted[group_A_indices_test] = (df_proba_group_A['Prob_Class_1'] >= best_threshold_A).astype(int)

# Apply threshold for Group B
group_B_indices_test = df.loc[y_test.index, 'Sensitive_Attribute'] == 1
y_pred_adjusted[group_B_indices_test] = (df_proba_group_B['Prob_Class_1'] >= best_threshold_B).astype(int)

metrics_table_adjusted = create_metrics_table(y_test, y_pred_adjusted, df, group_col_name='Group')
display(metrics_table_adjusted)

## 4a estratégia: paridade de erro ou acurácia

Por exemplo, podemos tentar equalizar a taxa de verdadeiros positivos entre grupos:


In [ ]:
# Function to calculate True Positive Rate (TPR) for a given threshold and group
def calculate_tpr(df, threshold):
    df_test = df.copy()
    df_test['Predicted_Label'] = (df_test['Prob_Class_1'] >= threshold).astype(int)
    true_positives = df_test[(df_test['Predicted_Label'] == 1) & (df_test['True_Label'] == 1)].shape[0]
    condition_positives = df_test[df_test['True_Label'] == 1].shape[0]
    tpr = true_positives / condition_positives if condition_positives > 0 else 0
    return tpr

def calculate_accuracy(df, threshold):
    df_test = df.copy()
    df_test['Predicted_Label'] = (df_test['Prob_Class_1'] >= threshold).astype(int)
    accuracy = (df_test['Predicted_Label'] == df_test['True_Label']).mean()
    return accuracy

# Function to find thresholds for equal opportunity while balancing accuracy
def find_thresholds_balanced_equal_opportunity(df_group_A, df_group_B, lambda_weight=0.5):
    best_threshold_A = 0
    best_threshold_B = 0
    best_combined_metric = float('inf')

    # Test thresholds for both groups
    for threshold_A in np.linspace(0, 1, 101):
        for threshold_B in np.linspace(0, 1, 101):
            # Calculate TPR for both groups
            tpr_A = calculate_tpr(df_group_A, threshold_A)
            tpr_B = calculate_tpr(df_group_B, threshold_B)
            tpr_diff = abs(tpr_A - tpr_B)

            # Calculate accuracy for both groups
            accuracy_A = calculate_accuracy(df_group_A, threshold_A)
            accuracy_B = calculate_accuracy(df_group_B, threshold_B)
            overall_accuracy = (accuracy_A + accuracy_B) / 2

            # Combine TPR difference and accuracy into a single metric
            combined_metric = lambda_weight * tpr_diff - (1 - lambda_weight) * overall_accuracy

            # Update best thresholds if the combined metric improves
            if combined_metric < best_combined_metric:
                best_combined_metric = combined_metric
                best_threshold_A = threshold_A
                best_threshold_B = threshold_B

    return best_threshold_A, best_threshold_B

# Find thresholds balancing equal opportunity and accuracy
best_threshold_A, best_threshold_B = find_thresholds_balanced_equal_opportunity(
    df_proba_group_A, df_proba_group_B, lambda_weight=0.5
)

print(f"Threshold for Group A (Equal Opportunity): {best_threshold_A}")
print(f"Threshold for Group B (Equal Opportunity): {best_threshold_B}")

---


In [ ]:
fig = plot_probability_with_jitter_by_group(df_proba, thresholds=[best_threshold_A, best_threshold_B])
fig.show()

In [ ]:
# Get y_pred based on the group-specific thresholds
y_pred_adjusted = np.zeros_like(y_test) # Initialize with zeros (default class 0)

# Apply threshold for Group A
group_A_indices_test = df.loc[y_test.index, 'Sensitive_Attribute'] == 0
y_pred_adjusted[group_A_indices_test] = (df_proba_group_A['Prob_Class_1'] >= best_threshold_A).astype(int)

# Apply threshold for Group B
group_B_indices_test = df.loc[y_test.index, 'Sensitive_Attribute'] == 1
y_pred_adjusted[group_B_indices_test] = (df_proba_group_B['Prob_Class_1'] >= best_threshold_B).astype(int)

metrics_table_adjusted = create_metrics_table(y_test, y_pred_adjusted, df, group_col_name='Group')
display(metrics_table_adjusted)

## Visões sobre justiça (Barocas, Hardt, Narayanan) {.smaller}

| Visão             | Objetivo                                                                                                             | Intervenção                                                    | Quem arca com o custo                                   | Exemplos de técnicas e critérios           | Problemas                                                                          |
| ----------------- | -------------------------------------------------------------------------------------------------------------------- | -------------------------------------------------------------- | ------------------------------------------------------- | ------------------------------------------ | ----------------------------------------------------------------------------------------------- |
| **Estrita**       | Garantir que pessoas com **qualificações semelhantes** tenham **chances semelhantes** de acesso a oportunidades.     | No momento da **tomada de decisão**                            | **Ninguém**                                             | Ignorar grupos; maximizar lucro            | Ignora desigualdades históricas e estruturais; pressupõe que é possível medir o mérito.          |
| **Intermediária** | **Desconsiderar diferenças** que surgem de **injustiças passadas**, ajustando decisões para compensar desigualdades. | Na **tomada de decisão**, especialmente em momentos críticos   | **Tomador de decisão** (pode repassar custo ao afetado) | Paridade de erro, acurácia, outros ajustes | Falta de critérios claros; risco de injustiça reversa; difícil quantificar compensações.        |
| **Ampla**         | Garantir que pessoas com **habilidades e ambições semelhantes** possam **realizar seu potencial igualmente**.        | **Ação contínua do Estado** (educação, saúde, habitação, etc.) | **Contribuintes**                                       | Paridade demográfica; políticas públicas   | Exige mudanças estruturais; alto custo político e econômico; difícil de aplicar no curto prazo. |

## Outras abordagens (talvez tentando se aproximar do meio)...

---

### Equidade por indivíduo

- Em vez de agrupar as pessoas por categorias como gênero ou raça, avalia-se semelhança entre indivíduos com base em seus atributos relevantes (como desempenho, habilidades, contexto social)
- Garantir que não haja discriminação entre aqueles que são equivalentes segundo esse critério.

Desafios:

- Como definir “semelhança”?
- Quais atributos são relevantes para comparar indivíduos?
- Ignorar completamente as identidades coletivas (como raça e gênero) pode, paradoxalmente, apagar as injustiças sistemáticas vividas por grupos historicamente marginalizados.

---

### Equidade por contrafactuais

"Se esta mesma pessoa não fosse do gênero X, ou da cor Y, ou da idade Z, o resultado seria diferente?"

Desafios:

- É possível saber como seria o “mundo alternativo”?
- Como simular a mesma pessoa com tudo igual, exceto um aspecto identitário fundamental?
- Esbarra em limites filosóficos e técnicos da modelagem de identidades humanas...

---

### Equidade por causalidade

Uso de modelos causais para avaliar se o atributo sensível está realmente influenciando o resultado de forma indevida.

Desafios: 

- Conseguimos modelar corretamente todas as causas relevantes?
- Quais variáveis ocultas não estamos considerando?
- Bem mais sofisticado e difícil de levar adiante.

---

- Por exemplo, *path analysis*, *do-calculus*

![](causal-ex.svg)

## O problema do mérito {.smaller}

> "All Americans have not only the right, but the solemn responsibility to ascend as far as their talents and determination will take them." - Bill Clinton, 1995

> "Equidade significa poder competir apenas em torno daquilo que temos controle ou responsabilidade".

Parece justo, mas é extremamente **problemático**...

---

1. Não é simples identificar o que está e o que não está sob o controle de um indivíduo.
    - Por exemplo, o caso de uma gravidez: a pessoa tem certo controle, mas, no geral, concordamos que isso ainda não deveria afetar seus méritos.

2. Também não é simples **medir** o que está e o que não está sob o controle de um indivíduo

---

3. Coloca um peso infinito sobre o indivíduo

:::: {.columns}

::: {.column width="40%"}
![](sandel.jpg)
:::

::: {.column width="60%"}
![](future.jpg)
:::

::::

---

4. Faz os "bem-sucedidos" ignorarem a influência de fatores fora de seu controle que garantiram seus lugares.

> "Pessoas que, com um pouco de esforço e talento, vencem em uma meritocracia, carregam uma dívida pessoal que é ofuscada pela competição. [...] Dessa forma, mesmo uma meritocracia justa — em que não há trapaça, suborno ou privilégios especiais para os ricos — induz uma impressão equivocada: a de que chegamos onde estamos por conta própria. Os anos de trabalho árduo exigidos dos candidatos a universidades de elite praticamente os forçam a acreditar que seu sucesso é resultado exclusivo de suas próprias ações — e, se falharem, não têm a quem culpar senão a si mesmos. Esse é um fardo pesado para os jovens carregarem. Além disso, isso corrói as sensibilidades cívicas. Pois, quanto mais nos pensamos como pessoas autossuficientes e conquistadoras de si mesmas, mais difícil será aprender a gratidão e a humildade. E sem esses sentimentos, é difícil se importar com o bem comum." (Sandel, p. 23)

---

5. Cria uma cultura de violência

- Uma cultura de esforço para "subida social" formenta sentimentos de **ganância e possessividade**
    - O que gera também o pavor de ter sua própria posição/bens roubados
    - E uma ansiedade em justificar porque uns merecem algo e outros não...
        - Especialmente usando estatística ("preconceito formalizado")
- Assim, fomenta o discurso de ódio entre grupos
    - Justiça passa a ser "garantir que perdedores reconheçam o lugar deles e fiquem lá"
    - Teorias de bode expiatório: "o mundo está errado por causa de grupos que não estão onde deveriam estar"
    - Ex.: linchamentos públicos nos EUA entre 1883 e 1941

## O problema da otimização

* Normalmente, pessoas querem "maximizar a utilidade/eficiência", então acham que precisam escolher **A MELHOR** pessoa (independentemente de visões sobre justiça).

  * Assim, acaba-se confiando em métodos algoritmos complexos ([veja este exemplo](https://fairmlbook.org/legitimacy.html)).
  * No entanto, será que é sempre possível identificar **A MELHOR**?

* Nessa tentativa de maximização, **os processos de tomada de decisão frequentemente acabam recorrendo a critérios e medições infundadas**.

  * Divinação estatística”, apofenia...

* E se **não houver uma pessoa claramente melhor**? E se todos forem suficientemente qualificados?

  * **Randomização e sorteio**: um ponto defendido tanto por BHN quanto por Sandel (com as devidas e cuidadosas considerações, é claro).

    * Há um argumento forte de que, **às vezes, a randomização é a melhor forma de evitar discriminação**. Reconhecer os limites do nosso conhecimento e abrir espaço para a incerteza é uma das melhores maneiras de evitar olhar para aquilo que não deveríamos considerar.


## Uma outra perspectiva: além da escassez, da competição e do mérito

Walter Brueggemann, ["The Liturgy of Abundance, The Myth of Scarcity"](https://www.religion-online.org/article/the-liturgy-of-abundance-the-myth-of-scarcity/)

- E se o ponto de partida fosse de que há abundância no mundo e não precisamos competir em tudo?

- E se a ideia de "boa vida" significasse mais do que o bem individual, mas também dependesse do bem de todos (bem comum)?

> "Para muitos de nós, nosso primeiro pensamento ao considerar um sistema do qual fazemos parte é: 'Como posso chegar ao topo?', em vez de: 'Como posso ajudar aqueles que estão na base?'"


## O princípio da solidariedade

> "A solidariedade é uma palavra que com muita frequência é esquecida ou silenciada, porque é incômoda... Ela implica criar uma nova mentalidade, que pense em termos de comunidade e da prioridade da vida de todos sobre a apropriação dos bens por parte de poucos."
— Fratelli Tutti, §116, Papa Francisco

Enquanto a meritocracia enfatiza a ascensão individual e o merecimento, a solidariedade enfatiza:

 - A natureza compartilhada da vida e das oportunidades. ("Destino Universal dos Bens", Compêndio, §171);
 - A responsabilidade dos fortes para com os fracos, e opção preferencial pelos pobres;
 - Que a graça, e não o mérito, é a postura fundamental.

---

> "[Na perspectiva do Antigo Testamento] Os justos (ṣaddîq) estão dispostos a se colocar em desvantagem para beneficiar a comunidade; os ímpios estão dispostos a colocar a comunidade em desvantagem para beneficiar a si mesmos. Por isso, podemos dizer que a generosidade não é apenas uma questão de misericórdia, mas de justiça. Ezequiel 18:5 começa com uma afirmação geral de que o [justo], o ṣaddîq, pratica a justiça, e apresenta uma lista de onze formas concretas pelas quais os seres humanos exercem essas qualidades. Deixar de praticar qualquer uma dessas coisas é ser injusto, e a lista inclui 'partilhar o pão com o faminto e vestir o que está sem roupa' (v. 7). Portanto, não ser uma pessoa generosa é ser uma pessoa injusta."
  Tim Keller, "Justiça na Bíblia"

## Ideias? {.smaller}

E se repensássemos nossos modelos não como filtros de mérito, mas como mediações de cuidado?

| Pergunta tradicional (filtro de mérito) | Nova pergunta (mediação de cuidado)                          | O que isso mudaria na prática do sistema                                                           |
| --------------------------------------- | ------------------------------------------------------------ | -------------------------------------------------------------------------------------------------- |
| **Quem é o melhor?**                    | **Quem está precisando de apoio?**                           | Mudança do foco de performance para vulnerabilidade; introdução de variáveis de necessidade        |
| **Quem merece essa oportunidade?**      | **Como garantir que todos tenham acesso digno?**             | Uso de políticas de inclusão, reserva, sorteio justo quando há empate ou incerteza                 |
| **Como evitar que o sistema erre?**     | **Como minimizar sofrimento causado por decisões injustas?** | Sistemas com canais de recurso, explicabilidade, reversibilidade; preocupação com impacto social   |
| **Como prever com máxima precisão?**    | **Como agir com justiça mesmo quando a previsão falha?**     | Adoção de zonas de incerteza, human-in-the-loop, decisões prudenciais em vez de puramente técnicas |